Находим актуальные данные

In [4]:
from dateutil.parser import parse
from pathlib import Path

temperatures_path = Path.cwd()/'..'/'data'/'temperatures'
last_fetch_date = sorted(parse(fetch_date.stem) for fetch_date in temperatures_path.iterdir())[-1]
last_fetch_path = temperatures_path/last_fetch_date.isoformat()
last_fetch_path

PosixPath('/home/naheridi/DataspellProjects/temperature-analysis/notebooks/../data/temperatures/2021-04-12T13:38:31')

Формируем один DataFrame для удобства

In [5]:
from pandas import read_csv, DataFrame, to_numeric

path_to_raw = last_fetch_path/'1_raw'

temperatures = DataFrame([
    read_csv(
        path_to_raw/filename,
        header=None,
        names=[filename.stem],
        index_col=0,
        squeeze=True,
        parse_dates=True
    ).apply(
        to_numeric,
        errors='coerce',
        downcast='float'
    ).resample(
        '10Min'
    ).mean()
    for filename in path_to_raw.iterdir()
]).T
temperatures.dropna().head()

,wall_316,wall_210,bat_412_a,bat_316,bat_420,wall_412_a,bat_210,wall_420
2020-02-29 16:00:00,23.50,20.25,44.81,48.94,51.50,25.25,42.06,25.06
2020-02-29 16:10:00,23.44,20.56,44.88,48.94,51.75,25.31,42.25,25.25
2020-02-29 16:30:00,23.50,21.37,44.94,48.94,51.81,25.37,44.00,25.31
2020-02-29 16:50:00,23.50,21.75,44.81,49.13,51.88,25.44,44.13,25.06
2020-02-29 17:00:00,23.50,21.81,44.88,49.13,52.19,25.44,44.25,25.37


Сохраняем вместе с изначальными данными

In [6]:
normalized_path = last_fetch_path/'2_normalized'

normalized_path.mkdir(parents=True)

dataset_path = normalized_path/'dataset.csv'

temperatures.to_csv(dataset_path)
dataset_path

PosixPath('/home/naheridi/DataspellProjects/temperature-analysis/notebooks/../data/temperatures/2021-04-12T13:38:31/2_normalized/dataset.csv')